#  问题：利用torch.nn实现 softmax 回归在Fashion-MNIST数据集上进行训练和测试，并从loss，训练集以及测试集上的准确率等多个角度对结果进行分析


利用 torch.nn 实现 Softmax 回归模型在 Fashion-MNIST 数据集上进行训练和测试可以通过高层次的模块更加方便地进行操作。接下来将分步骤介绍如何使用 torch.nn 实现 Softmax 回归模型。

# 数据集加载与预处理
使用 torchvision 加载 Fashion-MNIST 数据集，并对数据进行标准化处理。

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 数据预处理，标准化到 [-1, 1] 之间
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# 加载训练和测试集
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# 构建 Softmax 回归模型
使用 torch.nn 来定义 Softmax 回归模型，这里使用一个全连接层来实现线性变换，再通过 softmax 函数进行归一化。

In [7]:
class SoftmaxRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)  # 输入层和输出层

    def forward(self, x):
        out = self.linear(x)
        return out  # 不需要手动加 softmax，后面会结合损失函数自动处理

# 初始化模型
input_size = 28 * 28  # 图片尺寸 28x28
num_classes = 10  # Fashion-MNIST 数据集有 10 个类别
model = SoftmaxRegression(input_size, num_classes)

# 定义损失函数和优化器
使用交叉熵损失函数 nn.CrossEntropyLoss()，它会自动将 logits 应用 softmax。优化器使用随机梯度下降（SGD）。

In [8]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 交叉熵损失
optimizer = optim.SGD(model.parameters(), lr=0.01)  # 随机梯度下降优化器

# 训练模型
在训练过程中，会将每个批次的数据送入模型，计算损失，并通过反向传播更新参数。

In [9]:
# 训练模型
num_epochs = 100  # 设置训练的次数

for epoch in range(num_epochs):
    total_loss = 0
    total_correct = 0
    
    for images, labels in train_loader:
        # 将图片展平成一维向量
        images = images.view(-1, 28*28)
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()  # 梯度清零
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        
        # 计算准确率
        _, predicted = torch.max(outputs.data, 1)  # 找出最大值所对应的类别
        total_correct += (predicted == labels).sum().item()
        total_loss += loss.item()
    
    accuracy = total_correct / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.4f}')

Epoch [1/100], Loss: 0.6795, Accuracy: 0.7760
Epoch [2/100], Loss: 0.5177, Accuracy: 0.8218
Epoch [3/100], Loss: 0.4851, Accuracy: 0.8335
Epoch [4/100], Loss: 0.4681, Accuracy: 0.8390
Epoch [5/100], Loss: 0.4560, Accuracy: 0.8434
Epoch [6/100], Loss: 0.4477, Accuracy: 0.8467
Epoch [7/100], Loss: 0.4411, Accuracy: 0.8486
Epoch [8/100], Loss: 0.4354, Accuracy: 0.8508
Epoch [9/100], Loss: 0.4315, Accuracy: 0.8516
Epoch [10/100], Loss: 0.4272, Accuracy: 0.8535
Epoch [11/100], Loss: 0.4243, Accuracy: 0.8539
Epoch [12/100], Loss: 0.4215, Accuracy: 0.8555
Epoch [13/100], Loss: 0.4191, Accuracy: 0.8565
Epoch [14/100], Loss: 0.4170, Accuracy: 0.8564
Epoch [15/100], Loss: 0.4148, Accuracy: 0.8568
Epoch [16/100], Loss: 0.4127, Accuracy: 0.8582
Epoch [17/100], Loss: 0.4111, Accuracy: 0.8586
Epoch [18/100], Loss: 0.4095, Accuracy: 0.8591
Epoch [19/100], Loss: 0.4078, Accuracy: 0.8596
Epoch [20/100], Loss: 0.4066, Accuracy: 0.8597
Epoch [21/100], Loss: 0.4054, Accuracy: 0.8608
Epoch [22/100], Loss: 

# 测试模型
训练完成后，在测试集上评估模型，计算损失和准确率。

In [10]:
# 测试模型
def test(model, test_loader):
    model.eval()  # 切换到评估模式
    total_correct = 0
    total_loss = 0
    
    with torch.no_grad():  # 在测试时不需要计算梯度
        for images, labels in test_loader:
            images = images.view(-1, 28*28)
            outputs = model(images)
            
            # 计算损失
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            # 计算准确率
            _, predicted = torch.max(outputs.data, 1)
            total_correct += (predicted == labels).sum().item()
    
    accuracy = total_correct / len(test_loader.dataset)
    print(f'Test Loss: {total_loss/len(test_loader):.4f}, Test Accuracy: {accuracy:.4f}')

# 测试模型
test(model, test_loader)

Test Loss: 0.4383, Test Accuracy: 0.8410


# 分析结果
+ Loss: 训练过程中的损失会逐渐下降，表明模型在学习过程中表现良好。
+ Accuracy: 训练集和测试集的准确率逐渐提高，表明模型对数据的分类能力在不断增强。

# 总结
通过使用 torch.nn 实现 Softmax 回归模型，我在 Fashion-MNIST 数据集上进行了训练和测试。我从损失、训练集准确率和测试集准确率等多个角度分析了模型的表现。通过这种方式，可以快速搭建出一个高效的分类模型。